# Notebook Imports

In [1]:
import pandas as pd
import numpy as np

# Constants

In [3]:
TRAINING_DATA_FILE = 'C:/Users/rachi/Downloads/train-data2.txt'
TEST_DATA_FILE = 'C:/Users/rachi/Downloads/test-data2.txt'

VOCAB_SIZE = 2500

# Read and Load Features from .txt Files into NumPy Array

In [4]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter=' ', dtype = int)

In [5]:
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter=' ', dtype = int)

In [6]:
sparse_train_data[:5]

array([[ 0,  2,  0,  1],
       [ 0,  5,  0,  1],
       [ 0,  6,  0,  1],
       [ 0,  8,  0,  1],
       [ 0, 13,  0,  1]])

In [7]:
sparse_train_data[-5:]

array([[5165, 1474,    0,    1],
       [5165, 1518,    0,    1],
       [5165, 1753,    0,    1],
       [5165, 1868,    0,    1],
       [5165, 2017,    0,    1]])

In [8]:
print('Number of rows in training file: ', sparse_train_data.shape[0])
print('Number of rows in test file: ', sparse_test_data.shape[0])

Number of rows in training file:  157437
Number of rows in test file:  67139


In [9]:
print('Number of emails in training file:', np.unique(sparse_train_data[:,0]).size)

Number of emails in training file: 3600


In [10]:
print('Number of emails in test file:', np.unique(sparse_test_data[:,0]).size)

Number of emails in test file: 1546


# Create a Empty DataFrame

In [11]:
column_names=['DOC_ID']+['CATEGORY']+list(range(0,VOCAB_SIZE))
column_names[:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [12]:
len(column_names)

2502

In [13]:
index_names = np.unique(sparse_train_data[:, 0])
index_names

array([   0,    1,    2, ..., 5162, 5163, 5165])

In [14]:
full_train_data = pd.DataFrame(index = index_names, columns = column_names)
full_train_data.fillna(value=0, inplace = True)

C:\Users\rachi\AppData\Local\Temp\ipykernel_4580\2370846799.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  full_train_data.fillna(value=0, inplace = True)


In [15]:
full_train_data.head()

DOC_ID  CATEGORY  0  1  2  3  4  5  6  7  ...  2490  2491  2492  2493  \
0       0         0  0  0  0  0  0  0  0  0  ...     0     0     0     0   
1       0         0  0  0  0  0  0  0  0  0  ...     0     0     0     0   
2       0         0  0  0  0  0  0  0  0  0  ...     0     0     0     0   
3       0         0  0  0  0  0  0  0  0  0  ...     0     0     0     0   
4       0         0  0  0  0  0  0  0  0  0  ...     0     0     0     0   

   2494  2495  2496  2497  2498  2499  
0     0     0     0     0     0     0  
1     0     0     0     0     0     0  
2     0     0     0     0     0     0  
3     0     0     0     0     0     0  
4     0     0     0     0     0     0  

[5 rows x 2502 columns]

# Create a Full Matrix from a Sparse Matrix

In [16]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    """
    Form a full matrix from a sparse matrix. Return a pandas dataframe.
    Keyword arguments:
    sparse_matrix -- numpy array
    nr_words -- size of the vocabulary. Total number of tokens.
    doc_idx -- position of the document id in the sparse matrix. Default: 1st column
    word_idx -- position of the word id in the sparse matrix. Default: 2nd column
    cat_idx -- position of the label (spam is 1, nonspam is 0). Default: 3rd column
    freq_idx -- position of occurrence of word in sparse matrix. Default: 4th column
    """
    
    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
    doc_id_names = np.unique(sparse_matrix[:, 0])
    full_matrix = pd.DataFrame(index = doc_id_names, columns = column_names)
    full_matrix.fillna(value=0, inplace = True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurrence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr, 'CATEGORY'] = label
        full_matrix.at[doc_nr, word_id] = occurrence
    
    full_matrix.set_index('DOC_ID', inplace = True)
    return full_matrix

In [17]:
%%time

full_train_data=make_full_matrix(sparse_train_data, VOCAB_SIZE)

C:\Users\rachi\AppData\Local\Temp\ipykernel_4580\630567689.py:16: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  full_matrix.fillna(value=0, inplace = True)


CPU times: total: 6.98 s
Wall time: 12 s


In [18]:
full_train_data.head()

CATEGORY  0  1  2  3  4  5  6  7  8  ...  2490  2491  2492  2493  \
DOC_ID                                       ...                           
0              0  0  0  1  0  0  1  1  0  1  ...     0     0     0     0   
1              0  0  0  0  0  0  0  0  0  0  ...     0     0     0     0   
2              0  0  0  0  1  0  0  0  0  0  ...     0     0     0     0   
3              1  0  0  0  0  0  0  0  0  0  ...     0     0     0     0   
4              0  0  0  0  0  3  0  0  0  0  ...     0     0     0     0   

        2494  2495  2496  2497  2498  2499  
DOC_ID                                      
0          0     0     0     0     0     0  
1          0     0     0     0     0     0  
2          0     0     0     0     0     0  
3          0     0     0     0     0     0  
4          0     0     0     0     0     0  

[5 rows x 2501 columns]

In [19]:
full_train_data.tail()

CATEGORY   0   1   2  3  4  5  6   7   8  ...  2490  2491  2492  2493  \
DOC_ID                                            ...                           
5158           0   0   0   4  0  0  0  2   1   1  ...     0     0     0     0   
5160           0   0   0   0  0  0  3  0   0   0  ...     0     0     0     0   
5162           1   0   0   0  0  0  0  0   0   0  ...     0     0     0     0   
5163           0  67  36  17  0  2  3  5  10  22  ...     0     0     0     0   
5165           0   0   0   0  0  0  1  1   1   2  ...     0     0     0     0   

        2494  2495  2496  2497  2498  2499  
DOC_ID                                      
5158       0     0     0     0     0     0  
5160       0     0     0     0     0     0  
5162       0     0     0     0     0     0  
5163       0     0     0     0     0     0  
5165       0     0     0     0     0     0  

[5 rows x 2501 columns]

# Training the Naive Bayes Model
`
## Calculating the Probability of Spam

In [21]:
prob_spam = full_train_data.CATEGORY.sum() / full_train_data.CATEGORY.size
print('Probability of spam is: ', prob_spam)

Probability of spam is:  0.29194444444444445


# Total Number of  Words/Tokens

In [23]:
full_train_features = full_train_data.loc[:, full_train_data.columns != 'CATEGORY']
full_train_features.head()

0     1     2     3     4     5     6     7     8     9     ...  2490  \
DOC_ID                                                              ...         
0          0     0     1     0     0     1     1     0     1     0  ...     0   
1          0     0     0     0     0     0     0     0     0     0  ...     0   
2          0     0     0     1     0     0     0     0     0     0  ...     0   
3          0     0     0     0     0     0     0     0     0     0  ...     0   
4          0     0     0     0     3     0     0     0     0     0  ...     0   

        2491  2492  2493  2494  2495  2496  2497  2498  2499  
DOC_ID                                                        
0          0     0     0     0     0     0     0     0     0  
1          0     0     0     0     0     0     0     0     0  
2          0     0     0     0     0     0     0     0     0  
3          0     0     0     0     0     0     0     0     0  
4          0     0     0     0     0     0     0     0     0  

[5 rows x 2500 columns]

In [24]:
email_lengths = full_train_features.sum(axis=1)
email_lengths.shape

(3600,)

In [25]:
email_lengths[:5]

DOC_ID
0     29
1     10
2    172
3     12
4     26
dtype: int64

In [26]:
total_wc = email_lengths.sum()
total_wc

np.int64(271958)

# Number of Tokens in Spam and Ham Emails

In [27]:
spam_lengths = email_lengths[full_train_data.CATEGORY == 1]
spam_lengths.shape

(1051,)

In [28]:
spam_wc = spam_lengths.sum()
spam_wc

np.int64(79119)

In [29]:
ham_lengths = email_lengths[full_train_data.CATEGORY == 0]
ham_lengths.shape

(2549,)

In [30]:
nonspam_wc = ham_lengths.sum()
nonspam_wc

np.int64(192839)

In [31]:
email_lengths.shape[0] - spam_lengths.shape[0] - ham_lengths.shape[0]

0

In [32]:
spam_wc + nonspam_wc - total_wc

np.int64(0)

# Summing the Tokens occuring in Spam

In [33]:
full_train_features.shape

(3600, 2500)

In [38]:
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY == 1]
train_spam_tokens.head()

0     1     2     3     4     5     6     7     8     9     ...  2490  \
DOC_ID                                                              ...         
3          0     0     0     0     0     0     0     0     0     0  ...     0   
7          0     0     0     1     0     0     0     0     0     0  ...     0   
10         0     0     0     0     0     2     0     0     0     0  ...     0   
11         0     0     0     0     0     0     0     0     0     0  ...     0   
13         0     0     0     0     0     0     0     0     0     0  ...     0   

        2491  2492  2493  2494  2495  2496  2497  2498  2499  
DOC_ID                                                        
3          0     0     0     0     0     0     0     0     0  
7          0     0     0     0     0     0     0     0     0  
10         0     0     0     0     0     0     0     0     0  
11         0     0     0     0     0     0     0     0     0  
13         0     0     0     0     0     0     0     0     0  

[5 rows x 2500 columns]

In [39]:
train_spam_tokens.tail()

0     1     2     3     4     5     6     7     8     9     ...  2490  \
DOC_ID                                                              ...         
5144       0     0     0     1     0     0     0     0     0     0  ...     0   
5148       0     0     0     0     0     0     0     0     0     0  ...     0   
5149       0     0     0     0     0     0     0     0     0     0  ...     0   
5151       0     0     0     0     0     0     0     0     0     0  ...     0   
5162       0     0     0     0     0     0     0     0     0     0  ...     0   

        2491  2492  2493  2494  2495  2496  2497  2498  2499  
DOC_ID                                                        
5144       0     0     0     0     0     0     0     0     0  
5148       0     0     0     0     0     0     0     0     0  
5149       0     0     0     0     0     0     0     0     0  
5151       0     0     0     0     0     0     0     0     0  
5162       0     0     0     0     0     0     0     0     0  

[5 rows x 2500 columns]

In [40]:
train_spam_tokens.shape

(1051, 2500)

In [41]:
summed_spam_tokens = train_spam_tokens.sum(axis = 0) + 1
summed_spam_tokens.shape

(2500,)

In [42]:
summed_spam_tokens.tail()

2495    11
2496    14
2497     1
2498    11
2499     7
dtype: int64

# Summing the Token occuring in Ham

In [43]:
train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY == 0]
summed_ham_tokens = train_ham_tokens.sum(axis=0) + 1

In [44]:
summed_ham_tokens.shape

(2500,)

In [45]:
summed_ham_tokens.tail()

2495     8
2496     5
2497    19
2498     7
2499     8
dtype: int64

In [46]:
summed_ham_tokens[2499].sum()+1

np.int64(9)

# P(Token | Spam) - Probability that a Token occurs given that the Email is Spam

In [47]:
prob_tokens_spam = summed_spam_tokens / (spam_wc + VOCAB_SIZE)
prob_tokens_spam[:5]

0    0.000135
1    0.000110
2    0.000012
3    0.009263
4    0.000956
dtype: float64

In [48]:
prob_tokens_spam.sum()

np.float64(1.0)

# P(Token | Ham) - Probability that a Token occurs given that the Email is Non-Spam

In [51]:
prob_tokens_nonspam = summed_ham_tokens / (nonspam_wc + VOCAB_SIZE)
prob_tokens_nonspam.sum()

np.float64(1.0)

# P(Token) - Probability that Token occurs

In [52]:
prob_tokens_all = full_train_features.sum(axis=0) / total_wc
prob_tokens_all.sum()

np.float64(1.0)

# Saved the Trained Model

In [53]:
TOKEN_SPAM_PROB_FILE = "token_spam_prob2.txt"
TOKEN_HAM_PROB_FILE = "token_ham_prob2.txt"
TOKEN_ALL_PROB_FILE = "token_all_prob2.txt"

np.savetxt(TOKEN_SPAM_PROB_FILE, prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE, prob_tokens_nonspam)
np.savetxt(TOKEN_ALL_PROB_FILE, prob_tokens_all)

# Prepare Test Data

In [54]:
sparse_test_data.shape

(67139, 4)

In [55]:
%%time

full_test_data = make_full_matrix(sparse_test_data, nr_words=VOCAB_SIZE)

C:\Users\rachi\AppData\Local\Temp\ipykernel_4580\630567689.py:16: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  full_matrix.fillna(value=0, inplace = True)


CPU times: total: 3.2 s
Wall time: 5.95 s


In [56]:
X_test = full_test_data.loc[:, full_test_data.columns != 'CATEGORY']
y_test = full_test_data.CATEGORY

In [57]:
TEST_FEATURE_MATRIX = 'test-features2.txt'
TEST_TARGET_FILE = 'test-target2.txt'

np.savetxt(TEST_TARGET_FILE, y_test)
np.savetxt(TEST_FEATURE_MATRIX, X_test)